In [3]:
import pandas as pd

df = pd.read_csv('/KRX/data/fixed_sample_questions.csv')
df

,question,answer
0,"파생상품, 알고리즘 거래, 고주파 거래는 각각 금융시장에 다양한 영향을 미치는 요소...",3
1,"IMF와 세계은행이 경제 안정성에 기여하는 방식에서, 다음 중 가장 적절한 설명은 ...",1
2,"AI가 신용 평가 및 사기 탐지에 어떻게 활용되고 있는지를 논의할 때, 다음 중 어...",5
3,AI가 자산 관리에서 생성하는 가치를 극대화하기 위해 필요한 도전 과제를 고려할 때...,1
4,AI 기반 금융 서비스에서 고객 경험을 향상시키기 위한 여러 접근 방식이 있습니다....,4
...,...,...
994,"AI와 블록체인 기술의 도입이 도매 은행에 미치는 기회와 도전 과제를 고려할 때, ...",3
995,"회사의 현금 흐름 관리에서 필수적인 요소인 현금 집중, 현금 예측 및 현금 지출 기...",5
996,"기업이 현금 집중과 현금 지급 전략을 활용하여 유동성을 유지하려고 할 때, 이러한 ...",1
997,애플과 아마존의 자본 구조에서 발견되는 공통적 요소와 자금 조달 전략의 차이점에 기...,3


In [47]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), 'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [48]:
system_prompt ="""
You are a financial expert.
Read the following questions and choose the answer.
It could be multiple correct answers.
Just write a number or numbers.
"""

user_prompt="""
Question : {}
"""

In [20]:
temp_df = df.iloc[:10].copy()
temp_df

,question,answer
0,"파생상품, 알고리즘 거래, 고주파 거래는 각각 금융시장에 다양한 영향을 미치는 요소...",3
1,"IMF와 세계은행이 경제 안정성에 기여하는 방식에서, 다음 중 가장 적절한 설명은 ...",1
2,"AI가 신용 평가 및 사기 탐지에 어떻게 활용되고 있는지를 논의할 때, 다음 중 어...",5
3,AI가 자산 관리에서 생성하는 가치를 극대화하기 위해 필요한 도전 과제를 고려할 때...,1
4,AI 기반 금융 서비스에서 고객 경험을 향상시키기 위한 여러 접근 방식이 있습니다....,4
5,"AI가 금융 서비스의 고객 신용 위험 평가를 개선하는 데 있어, 고객이 경험하는 서...",2
6,"AI를 활용한 고객 서비스 향상과 관련하여, 다음 중 가장 적절한 설명은 무엇인가요...",1
7,"AI가 신용 평가와 투자 결정에 어떻게 기여하는지를 설명한 후, AI가 투자자에게 ...",3
8,"AI가 금융 분야에 도입됨에 따라, GDPR, FSMA, AML 규제와 같은 다양한...",1
9,"AI를 통한 자산 관리의 효율성을 높이기 위해 규제 당국, 금융 기관, 그리고 투자...",1


In [22]:
gpt_answer = []
for _, row_item in temp_df.iterrows():
    answer = int(row_item['answer'])
    question = str(row_item['question'])

    system_msg = {'role': 'system', 'content':system_prompt}
    user_msg = {'role':'user', 'content':user_prompt.format(question)}

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [system_msg, user_msg]
    )

    response_text = response.choices[0].message.content

    print(response_text)
    print("*" * 100)

    gpt_4o_answer = []
    for i in range(6):
        if str(i) in response_text:
            gpt_4o_answer.append(i)

    gpt_answer.append(gpt_4o_answer)

question      파생상품, 알고리즘 거래, 고주파 거래는 각각 금융시장에 다양한 영향을 미치는 요소...
answer                                                        3
gpt_answer                                               [3, 5]
Name: 0, dtype: object
3
****************************************************************************************************
question      IMF와 세계은행이 경제 안정성에 기여하는 방식에서, 다음 중 가장 적절한 설명은 ...
answer                                                        1
gpt_answer                                               [1, 5]
Name: 1, dtype: object
1, 5
****************************************************************************************************
question      AI가 신용 평가 및 사기 탐지에 어떻게 활용되고 있는지를 논의할 때, 다음 중 어...
answer                                                        5
gpt_answer                                            [1, 3, 5]
Name: 2, dtype: object
1, 3, 5
****************************************************************************************************
question      AI가 자산 관리에서 생성하는 가치를 극대

In [43]:
fixed_system_prompt = """
Your role is to correct the options for financial matters. You will be given the problem and the option number that you will need to rewrite. Please correct the given option number's sentence for your question by meeting the following conditions.
1. The selection must not deviate from the subject of the problem.
2. It should undoubtedly be corrected with a perfect incorrect answer.
3. It must not be modified in the same sense as other fingerprints.
4. The length selected must be similar to the length of other options.
You must only write correct options which satisfy above conditions.
"""

fixed_user_prompt="""
Question: {}
fixed_numbers : {}
"""

In [45]:
fixed_results = []
temp_df['gpt_answer'] = gpt_answer

for idx, row_item in temp_df.iterrows():
    question = row_item['question']
    answer = row_item['answer']
    gpt_answer = row_item['gpt_answer']
    fixed_question = ''

    need_to_fix_numbers = [num for num in gpt_answer if num != answer]
    if len(need_to_fix_numbers) >= 1:
        fixed_numbers_str = ', '.join(map(str, need_to_fix_numbers))
        fixed_system_msg = {'role':'system', 'content': fixed_system_prompt}
        fixed_user_msg={'role':'user', 'content': fixed_user_prompt.format(question, fixed_numbers_str)}

        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages = [fixed_system_msg, fixed_user_msg]
        )

        result = response.choices[0].message.content
        question_lines = question.split('\n')
        for fixed_choice in result.split('\n'):
            fixed_choice_text = fixed_choice.strip()
                
            for i in range(1, 6):
                if fixed_choice_text.startswith(str(i)):
                    question_lines[i] = fixed_choice_text

        fixed_question = '\n'.join(question_lines)

    if fixed_question:
        new_item = {'question':fixed_question, 'answer':answer}
    else:
        new_item = {'question':question, 'answer':answer}

    fixed_results.append(new_item)
            


In [46]:
fixed_df = pd.DataFrame(fixed_results)
fixed_df

,question,answer
0,"파생상품, 알고리즘 거래, 고주파 거래는 각각 금융시장에 다양한 영향을 미치는 요소...",3
1,"IMF와 세계은행이 경제 안정성에 기여하는 방식에서, 다음 중 가장 적절한 설명은 ...",1
2,"AI가 신용 평가 및 사기 탐지에 어떻게 활용되고 있는지를 논의할 때, 다음 중 어...",5
3,AI가 자산 관리에서 생성하는 가치를 극대화하기 위해 필요한 도전 과제를 고려할 때...,1
4,AI 기반 금융 서비스에서 고객 경험을 향상시키기 위한 여러 접근 방식이 있습니다....,4
5,"AI가 금융 서비스의 고객 신용 위험 평가를 개선하는 데 있어, 고객이 경험하는 서...",2
6,"AI를 활용한 고객 서비스 향상과 관련하여, 다음 중 가장 적절한 설명은 무엇인가요...",1
7,"AI가 신용 평가와 투자 결정에 어떻게 기여하는지를 설명한 후, AI가 투자자에게 ...",3
8,"AI가 금융 분야에 도입됨에 따라, GDPR, FSMA, AML 규제와 같은 다양한...",1
9,"AI를 통한 자산 관리의 효율성을 높이기 위해 규제 당국, 금융 기관, 그리고 투자...",1


# 전체 quality_control

In [50]:
from tqdm import tqdm

gpt_answer = []
for _, row_item in tqdm(df.iterrows(), total=len(df)):
    answer = int(row_item['answer'])
    question = str(row_item['question'])

    system_msg = {'role': 'system', 'content':system_prompt}
    user_msg = {'role':'user', 'content':user_prompt.format(question)}

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [system_msg, user_msg]
    )

    response_text = response.choices[0].message.content

    gpt_4o_answer = []
    for i in range(6):
        if str(i) in response_text:
            gpt_4o_answer.append(i)

    gpt_answer.append(gpt_4o_answer)

100%|██████████| 999/999 [12:38<00:00,  1.32it/s]


In [51]:
fixed_results = []
df['gpt_answer'] = gpt_answer

for idx, row_item in tqdm(df.iterrows(), total=len(df)):
    question = row_item['question']
    answer = row_item['answer']
    gpt_answer = row_item['gpt_answer']
    fixed_question = ''

    need_to_fix_numbers = [num for num in gpt_answer if num != answer]
    if len(need_to_fix_numbers) >= 1:
        fixed_numbers_str = ', '.join(map(str, need_to_fix_numbers))
        fixed_system_msg = {'role':'system', 'content': fixed_system_prompt}
        fixed_user_msg={'role':'user', 'content': fixed_user_prompt.format(question, fixed_numbers_str)}

        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages = [fixed_system_msg, fixed_user_msg]
        )

        result = response.choices[0].message.content
        question_lines = question.split('\n')
        for fixed_choice in result.split('\n'):
            fixed_choice_text = fixed_choice.strip()
                
            for i in range(1, 6):
                if fixed_choice_text.startswith(str(i)):
                    question_lines[i] = fixed_choice_text

        fixed_question = '\n'.join(question_lines)

    if fixed_question:
        new_item = {'question':fixed_question, 'answer':answer}
    else:
        new_item = {'question':question, 'answer':answer}

    fixed_results.append(new_item)
            


100%|██████████| 999/999 [14:19<00:00,  1.16it/s]


In [52]:
fixed_df = pd.DataFrame(fixed_results)
fixed_df


,question,answer
0,"파생상품, 알고리즘 거래, 고주파 거래는 각각 금융시장에 다양한 영향을 미치는 요소...",3
1,"IMF와 세계은행이 경제 안정성에 기여하는 방식에서, 다음 중 가장 적절한 설명은 ...",1
2,"AI가 신용 평가 및 사기 탐지에 어떻게 활용되고 있는지를 논의할 때, 다음 중 어...",5
3,AI가 자산 관리에서 생성하는 가치를 극대화하기 위해 필요한 도전 과제를 고려할 때...,1
4,AI 기반 금융 서비스에서 고객 경험을 향상시키기 위한 여러 접근 방식이 있습니다....,4
...,...,...
994,"AI와 블록체인 기술의 도입이 도매 은행에 미치는 기회와 도전 과제를 고려할 때, ...",3
995,"회사의 현금 흐름 관리에서 필수적인 요소인 현금 집중, 현금 예측 및 현금 지출 기...",5
996,"기업이 현금 집중과 현금 지급 전략을 활용하여 유동성을 유지하려고 할 때, 이러한 ...",1
997,애플과 아마존의 자본 구조에서 발견되는 공통적 요소와 자금 조달 전략의 차이점에 기...,3


In [66]:
import re

def preprocess_questions(question):
    question_lines = question.split('\n')

    number_to_eng = {
        '1. ': 'A. ',
        '2. ': 'B. ',
        '3. ': 'C. ',
        '4. ': 'D. ',
        '5. ': 'E. ',
    }

    new_question_lines = []
    for idx, line in enumerate(question_lines):
        new_line = ''
        for i in range(6):
            choice_string = f"{str(i)}. "
            if line.startswith(choice_string):
                new_line = re.sub(choice_string, number_to_eng[choice_string], line)
                break
        if new_line:
            new_question_lines.append(new_line)
        else:
            new_question_lines.append(line)
    
    question_text = "### 질문: " + new_question_lines[0]
    choice_text = "### 선택지:\n" + '\n'.join(new_question_lines[1:])
    result = question_text + "\n" + choice_text

    return result

def preprocess_answer(answer):
    number_to_eng = {
        1: 'A',
        2: 'B',
        3: 'C',
        4: 'D',
        5: 'E',
    }

    return number_to_eng[answer] 

In [68]:
fixed_df['question'] = fixed_df['question'].apply(preprocess_questions)
fixed_df['answer'] = fixed_df['answer'].apply(preprocess_answer)

In [69]:
fixed_df.to_csv('/KRX/outputs/fixed_sample_questions.csv', index=False)